In [101]:
import numpy as np
import pandas as pd
from westgate.flaml_model_core import LendingModel
from westgate.flaml_model_utils import load_model
from sklearn.metrics import classification_report
import json

In [140]:
base_folder = 'C:/Users/karim/Documents/westgate'
data_folder = base_folder + '/data/training'

In [181]:
accepted_2022_df = pd.read_csv(data_folder + '/loan_outcomes_2022.csv', encoding='latin')
refused_2022_df = pd.read_csv(data_folder + '/refusal2022.csv', encoding='latin')

C:\Users\karim\AppData\Local\Temp\ipykernel_11192\2821053411.py:1: DtypeWarning: Columns (741) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_2022_df = pd.read_csv(data_folder + '/loan_outcomes_2022.csv', encoding='latin')


In [194]:
refused_2022_df['Id'] = refused_2022_df['loginId']

In [141]:
accepted_2023_df = pd.read_csv(data_folder + '/janToNov2023LoansAccepted.csv')
refused_2023_df = pd.read_csv(data_folder + '/refusal2023_IF.csv')

C:\Users\karim\AppData\Local\Temp\ipykernel_11192\1357624890.py:1: DtypeWarning: Columns (394) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_2023_df = pd.read_csv(data_folder + '/janToNov2023LoansAccepted.csv')


In [195]:
accepted_df = accepted_2022_df
refused_df = refused_2022_df

In [139]:
year=2023
month=12
percentile = 20

In [183]:
accepted_df['gender'] = accepted_df['gender'].str.lower()

In [196]:
refused_df.rename(columns={'requestDate': 'request_date'}, inplace=True)

In [197]:
accepted_valid_requestdate = accepted_df[~accepted_df['request_date'].isna()]
refused_valid_requestdate = refused_df[~refused_df['request_date'].isna()]

In [145]:
year_col = accepted_valid_requestdate['request_date'].str[0:4].astype(int)
month_col = accepted_valid_requestdate['request_date'].str[5:7].astype(int)

In [198]:
#accepted_df = accepted_valid_requestdate[(year_col==year) & (month_col==month)]
accepted_final_df = accepted_valid_requestdate

In [147]:
year_col = refused_valid_requestdate['request_date'].str[0:4].astype(int)
month_col = refused_valid_requestdate['request_date'].str[5:7].astype(int)

In [214]:
#refused_df = refused_valid_requestdate[(year_col==year) & (month_col==month)]
refused_final_df = refused_valid_requestdate

In [215]:
accepted_final_df['refused'] = 0
refused_final_df['refused'] = 1

In [216]:
refused_final_df['recurring_deposits_90_days'] = (
    refused_final_df['recurring_deposits_current_month'] + 
    refused_final_df['recurring_deposits_previous_month'] +
    refused_final_df['recurring_deposits_2_months_ago']
)

refused_final_df['sum_micro_loans_60_days'] = (
    refused_final_df['sum_micro_loan_payments_current_month'] + 
    refused_final_df['sum_micro_loan_payments_previous_month'] 
)

refused_final_df['recurring_deposits_90_days'] = (
    refused_final_df['recurring_deposits_current_month'] + 
    refused_final_df['recurring_deposits_previous_month'] +
    refused_final_df['recurring_deposits_2_months_ago']
)

refused_final_df['sum_micro_loans_60_days'] = (
    refused_final_df['sum_micro_loan_payments_current_month'] + 
    refused_final_df['sum_micro_loan_payments_previous_month'] 
)

# %%
print('# loans before auto-refusal: ' + str(len(refused_final_df)))

refused_final_df = refused_final_df[refused_final_df['account_age_days'] >= 85]

refused_final_df = refused_final_df[refused_final_df['count_nsf_90_days'] <= 8]

refused_final_df = refused_final_df[refused_final_df['count_nsf_30_days'] <= 6]

refused_final_df = refused_final_df[refused_final_df['count_stop_payment_90_days'] <= 4]

print('# loans after auto-refusal: ' + str(len(refused_final_df)))

# loans before auto-refusal: 37398
# loans after auto-refusal: 28579


In [217]:
model_default = load_model('default', basefolder=base_folder + '/model_binaries/')

In [218]:
accepted_filtered_df = model_default.filter_df(accepted_final_df)

126 rows removed by threshold filtering
458 Rows with 'error' column not NA have been discarded.
5 Rows with 'Id' column NA have been discarded.
2 Rows with 'account_age_days' column not positive have been discarded.


In [219]:
df = pd.concat([accepted_filtered_df, refused_final_df]).reset_index()

In [220]:
len(df)

38086

In [221]:
model_refusal:LendingModel = load_model('refusal', basefolder=base_folder + '/model_binaries/')

In [222]:
pred_refusal = model_refusal.predict_proba(df, filter=True, engineer=True)

0 rows removed by threshold filtering
Error calculating age with dob ########## and request_date 2022-06-15 00:00:00
Unknown string format: ##########
Error calculating age with dob ########## and request_date 2022-09-14 00:00:00
Unknown string format: ##########
Error calculating age with dob ########## and request_date 2022-10-03 00:00:00
Unknown string format: ##########
Error calculating age with dob ########## and request_date 2022-10-06 00:00:00
Unknown string format: ##########


In [223]:
len(pred_refusal)

38086

In [224]:
df_pred = df.loc[pred_refusal.index]

In [225]:
len(df_pred)

38086

In [226]:
y_test = df_pred['refused']

In [227]:
y_pred = np.where(pred_refusal >= model_refusal.percentiles[percentile], 1, 0)

df_pred['y_test'] = y_test
df_pred['y_pred'] = y_pred

In [228]:
result = {
    'nb loans': len(df_pred),
    'nb_predicted_refusals': int(y_pred.sum()),
    'nb_actual_refusals': int(y_test.sum()),
    'pct_predicted_refusals': float(y_pred.sum() / y_pred.size),
    'classification_report': classification_report(y_test, y_pred, output_dict=True)
}


In [229]:
print(json.dumps(result, indent=4))

{
    "nb loans": 38086,
    "nb_predicted_refusals": 24464,
    "nb_actual_refusals": 28579,
    "pct_predicted_refusals": 0.6423357664233577,
    "classification_report": {
        "0": {
            "precision": 0.47364557333724855,
            "recall": 0.6786578310718419,
            "f1-score": 0.5579143067145143,
            "support": 9507.0
        },
        "1": {
            "precision": 0.8751226291693918,
            "recall": 0.7491164841317051,
            "f1-score": 0.8072318684840601,
            "support": 28579.0
        },
        "accuracy": 0.7315286456965814,
        "macro avg": {
            "precision": 0.6743841012533202,
            "recall": 0.7138871576017735,
            "f1-score": 0.6825730875992873,
            "support": 38086.0
        },
        "weighted avg": {
            "precision": 0.7749062144816801,
            "recall": 0.7315286456965814,
            "f1-score": 0.7449973975566047,
            "support": 38086.0
        }
    }
}
